# Space

In [1]:
import os
import sys 
import logging
import random
import pandas as pd 
from pprint import pprint 
from IPython.display import display, HTML

# WorkSpace
KEY = 'WorkSpace'; WORKSPACE_PATH = os.getcwd().split(KEY)[0] + KEY; print(WORKSPACE_PATH)
os.chdir(WORKSPACE_PATH)
sys.path.append(WORKSPACE_PATH)

# Pipeline Space
from proj_space import SPACE
SPACE['WORKSPACE_PATH'] = WORKSPACE_PATH
sys.path.append(SPACE['CODE_FN'])
pprint(SPACE)


# Available Packages
import argparse
import datasets
import pandas as pd
from datetime import datetime 

from recfldtkn.ckpd_obs import Ckpd_ObservationS
from recfldtkn.configfn import load_cohort_args
from recfldtkn.obsname import convert_RecObsName_and_CaseTkn_to_CaseObsName
from recfldtkn.loadtools import load_module_variables, update_args_to_list
from recfldtkn.observer import get_RecObsName_to_RecObsInfo, CaseObserverTransformer


logger = logging.getLogger(__name__)
recfldtkn_config_path = os.path.join(SPACE['CODE_RFT'], 'config_recfldtkn/')

/Users/jluo/Desktop/2024-DrFirst-SPACE/_DrFirst-AI-EduRxPred-WorkSpace
{'CODE_FN': '../pipeline',
 'CODE_RFT': '../pipeline',
 'DATA_CaseObs': '../_Data/2-Data_CaseObs',
 'DATA_RAW': '../_Data/0-Data_Raw',
 'DATA_RFT': '../_Data/1-Data_RFT',
 'DATA_TASK': 'Data_EduRxPred',
 'MODEL_TASK': 'Model_EduRxPred',
 'WORKSPACE_PATH': '/Users/jluo/Desktop/2024-DrFirst-SPACE/_DrFirst-AI-EduRxPred-WorkSpace'}


/Users/jluo/miniforge3/envs/torch/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# [Part 1] Args

## [Step 1] Args

In [2]:
##################################
my_parser = argparse.ArgumentParser(description='Process Input.')
# Add the arguments


my_parser.add_argument('--caseset_name',
                    metavar='caseset_name',
                    # nargs='+',  
                    default = None, 
                    type=str)

my_parser.add_argument('--splitset_name_list',
                    metavar='splitset_name_list',
                    nargs='+',  
                    default = None, 
                    type=str)

my_parser.add_argument('--subgroup_id_list',
                    metavar='subgroup_id_list',
                    nargs='+',  
                    default = None, 
                    type=str)

my_parser.add_argument('--subgroup_filter_fn',
                    metavar='subgroup_filter_fn',
                    default = None, 
                    type=str)

my_parser.add_argument('--case_id_columns',
                    metavar='case_id_columns',
                    nargs='+',  
                    default = None, 
                    type=str)

my_parser.add_argument('--case_observations',
                    metavar='case_observations',
                    nargs='+',  
                    default = None, 
                    type=str)

my_parser.add_argument('--case_taskop',
                    metavar='case_taskop',
                    default = None, 
                    type=str)

_StoreAction(option_strings=['--case_taskop'], dest='case_taskop', nargs=None, const=None, default=None, type=<class 'str'>, choices=None, required=False, help=None, metavar='case_taskop')

## [Step 2] Generate Arguments

In [3]:
argument_list = [

    # [Part 1] C: define my case set from a CaseSet.
    # ds_case_dict. 
    '--caseset_name', 
        'TrulicityRx',
    
    '--splitset_name_list', 
        'whole',
        # 'rs42-ds1-out0ts2023.11.01vd0.1-in_train',
        # 'rs42-ds1-out0ts2023.11.01vd0.1-in_valid',
        # 'rs42-ds1-out0ts2023.11.01vd0.1-in_test',

    '--subgroup_id_list', 
        'all', # '0', # 'all', # '0', '1', '2', '3', '4', # 

    '--subgroup_filter_fn', 
        'None', # 'subgroup_filter_fn',

    '--case_id_columns', 
        'PID', 'ObsDT', 'PInvID', 'RxID',
        
    # [Part 2] CO: case observations
    # CO = ro.ROList_ct.Phi
    # RO: record observations: R_i{RecName, Ckpd, phi(FldTkn)}
    # Phi: FutRxEduTkn
    '--case_observations', 
        'FutEduTknY:ro.EgmEdu-Af1W_ct.FutRxEduTkn',  
        'PDemo:ro.P-Demo_ct.InCaseTkn',
        'PZip3Demo:ro.P-Zip3DemoNume_ct.InCaseTkn',
        'PZip3Econ:ro.P-Zip3EconNume_ct.InCaseTkn',
        'PZip3House:ro.P-Zip3HousingNume_ct.InCaseTkn',
        'PZip3Social:ro.P-Zip3SocialNume_ct.InCaseTkn',
        # 'RxInCase1:ro.Rx-InObs-CmpCate_ct.InCaseTkn',
        # 'RxInCase2:ro.Rx-InObs-InsCate_ct.InCaseTkn',
        # 'RxInCase4:ro.Rx-InObs-QuantNume_ct.InCaseTkn',
        # 'RxInCase5:ro.Rx-InObs-ServiceCate_ct.InCaseTkn',
        # 'RxInCase6:ro.Rx-InObs-SysCate_ct.InCaseTkn',
        # 'RxInObsNum:ro.Rx-InObs_ct.RecNum',

    # [Part 3] Gamma: case task operations
    # case task operations: Gamma function.
    '--case_taskop', 
        'EduRxMLPred', # Edu Button Engagement Rx-Level ML Prediction
]

args = my_parser.parse_args(argument_list)

In [4]:
args = my_parser.parse_args(argument_list)
case_observations = args.case_observations
print(case_observations)

['FutEduTknY:ro.EgmEdu-Af1W_ct.FutRxEduTkn', 'PDemo:ro.P-Demo_ct.InCaseTkn', 'PZip3Demo:ro.P-Zip3DemoNume_ct.InCaseTkn', 'PZip3Econ:ro.P-Zip3EconNume_ct.InCaseTkn', 'PZip3House:ro.P-Zip3HousingNume_ct.InCaseTkn', 'PZip3Social:ro.P-Zip3SocialNume_ct.InCaseTkn']


# [Part 2] Case to Observe

## [Step 1]: Caseset + SplitType

In [5]:
from recfldtkn.configfn import load_cohort_args

caseset_name = args.caseset_name
print(caseset_name)
splitset_name_list = update_args_to_list(args.splitset_name_list)
print(splitset_name_list)

TrulicityRx
['whole']


In [6]:
case_type_name_list = []
for splitset_name in splitset_name_list:
    if 'rs' in splitset_name:
        case_type_name_list.append(caseset_name + '-' + splitset_name)
    else:
        case_type_name_list.append(caseset_name)
case_type_name_list

['TrulicityRx']

## [Step 2] SubGroup list

In [7]:
subgroup_id_list = update_args_to_list(args.subgroup_id_list)
if 'all' in subgroup_id_list: 
    group_num = len(os.listdir(os.path.join(SPACE['DATA_TASK'], 'CaseFolder', caseset_name)))
    subgroup_id_list = list(range(group_num))
else:
    subgroup_id_list = [int(i) for i in update_args_to_list(args.subgroup_id_list)]

subgroup_id_list

[0, 1, 2, 3, 4]

## [Step 3] Get ds_case_dict (rough version)

In [8]:
from recfldtkn.aidstools import get_caseset_to_observe


case_id_columns = update_args_to_list(args.case_id_columns) 

d = {}
for case_type in case_type_name_list:
    print('\n========================')
    print('case_type---->', case_type)

    # L = []
    for group_id in subgroup_id_list:
        print('group_id------->', group_id)
        CaseFolder = os.path.join(SPACE['DATA_TASK'], 'CaseFolder', case_type)
        print(CaseFolder)
        cohort_args = load_cohort_args(recfldtkn_config_path, SPACE)
        cohort_args['Ckpd_ObservationS'] = Ckpd_ObservationS
        print(cohort_args)
        group_name, ds_case = get_caseset_to_observe(group_id, CaseFolder, case_id_columns, cohort_args)
        print(group_name)
        print(ds_case)
        # L.append(ds_case)
        # ds_case_type = datasets.concatenate_datasets(L)
        d[f'{case_type}|{group_name}'] = ds_case

ds_case_dict = datasets.DatasetDict(d)
ds_case_dict


case_type----> TrulicityRx
group_id-------> 0
Data_EduRxPred/CaseFolder/TrulicityRx
{'RawRootID': 'patient_id_encoded', 'CohortInfo': {'RawData2023Nov_Trulicity': {'cohort_label': 1, 'cohort_name': 'RawData2023Nov_Trulicity', 'FolderPath': '../_Data/0-Data_Raw/2023_11_30_30days_Trulicity'}}, 'RootID': 'PID', 'RecName': 'PRawRecNum', 'rec_folder': '../_Data/1-Data_RFT/RecAttr/', 'fld_folder': '../_Data/1-Data_RFT/RecAttr/', 'hfds_folder': '../_Data/1-Data_RFT/HfDataset/', 'recattr_pyfolder': '../pipeline/fn_recattr/', 'fldtkn_pyfolder': '../pipeline/fn_fldtkn/', 'humanrec_pyfolder': '../pipeline/fn_humanrec/', 'RecName_to_RFT_GROUP_SIZE': {'Default': 5000}, 'RecName_to_RFT_idx_group_size': {'Default': 10000}, 'RecName_to_RFT_usebucket': {'Default': False}, 'RootIDLength': 6, 'recfldtkn_config_path': '../pipeline/config_recfldtkn/', 'SPACE': {'DATA_RAW': '../_Data/0-Data_Raw', 'DATA_RFT': '../_Data/1-Data_RFT', 'DATA_CaseObs': '../_Data/2-Data_CaseObs', 'CODE_FN': '../pipeline', 'CODE_R

DatasetDict({
    TrulicityRx|0_C1-gF-Ab50: Dataset({
        features: ['PID', 'ObsDT', 'PInvID', 'RxID'],
        num_rows: 40517
    })
    TrulicityRx|1_C1-gF-Bl50: Dataset({
        features: ['PID', 'ObsDT', 'PInvID', 'RxID'],
        num_rows: 17676
    })
    TrulicityRx|2_C1-gM-Ab50: Dataset({
        features: ['PID', 'ObsDT', 'PInvID', 'RxID'],
        num_rows: 32213
    })
    TrulicityRx|3_C1-gM-Bl50: Dataset({
        features: ['PID', 'ObsDT', 'PInvID', 'RxID'],
        num_rows: 10543
    })
    TrulicityRx|4_None: Dataset({
        features: ['PID', 'ObsDT', 'PInvID', 'RxID'],
        num_rows: 57
    })
})

## [Step 4] Filter Case Based on Case Labels

In [9]:
if args.subgroup_filter_fn == 'None': 
    ds_case_dict = ds_case_dict
    pass
else:
    ds_case_dict = ds_case_dict.filter(args.subgroup_filter_fn)

In [10]:
ds_case_dict

DatasetDict({
    TrulicityRx|0_C1-gF-Ab50: Dataset({
        features: ['PID', 'ObsDT', 'PInvID', 'RxID'],
        num_rows: 40517
    })
    TrulicityRx|1_C1-gF-Bl50: Dataset({
        features: ['PID', 'ObsDT', 'PInvID', 'RxID'],
        num_rows: 17676
    })
    TrulicityRx|2_C1-gM-Ab50: Dataset({
        features: ['PID', 'ObsDT', 'PInvID', 'RxID'],
        num_rows: 32213
    })
    TrulicityRx|3_C1-gM-Bl50: Dataset({
        features: ['PID', 'ObsDT', 'PInvID', 'RxID'],
        num_rows: 10543
    })
    TrulicityRx|4_None: Dataset({
        features: ['PID', 'ObsDT', 'PInvID', 'RxID'],
        num_rows: 57
    })
})

## [Step 5] get_ds_case_dict_from_settings

In [54]:
cohort_args = load_cohort_args(recfldtkn_config_path, SPACE)
cohort_args['Ckpd_ObservationS'] = Ckpd_ObservationS

In [55]:
def get_ds_case_dict_from_settings(caseset_name, 
                                   splitset_name_list, 
                                   subgroup_id_list, 
                                   subgroup_filter_fn, 
                                   cohort_args,
                                   case_id_columns = []):
    # caseset_name = args.caseset_name
    # print(caseset_name)
    # splitset_name_list = update_args_to_list(args.splitset_name_list)
    # print(splitset_name_list)
    SPACE = cohort_args['SPACE']
    case_type_name_list = []
    for splitset_name in splitset_name_list:
        if 'rs' in splitset_name:
            case_type_name_list.append(caseset_name + '-' + splitset_name)
        else:
            case_type_name_list.append(caseset_name)
    
    # case_type_name_list
    # case_id_columns = update_args_to_list(args.case_id_columns) 

    d = {}
    for case_type in case_type_name_list:
        # print('\n========================')
        # print('case_type---->', case_type)

        # L = []
        for group_id in subgroup_id_list:
            # print('group_id------->', group_id)
            CaseFolder = os.path.join(SPACE['DATA_TASK'], 'CaseFolder', case_type)
            # print(CaseFolder)
            
            if os.path.exists(CaseFolder) == False:
                print(f'CaseFolder does not exist: {CaseFolder}')
                print('TODO: ---- in the future, the Case Folder should be automatically created.')
                continue

            group_name, ds_case = get_caseset_to_observe(group_id, CaseFolder, case_id_columns, cohort_args)
            d[f'{case_type}|{group_name}'] = ds_case

    ds_case_dict = datasets.DatasetDict(d)
    
    if subgroup_filter_fn == 'None': 
        ds_case_dict = ds_case_dict
    else:
        # TODO
        ds_case_dict = ds_case_dict.filter(subgroup_filter_fn)
        
    return ds_case_dict

In [56]:
subgroup_filter_fn = args.subgroup_filter_fn

In [58]:
ds_case_dict = get_ds_case_dict_from_settings(caseset_name, 
                                   splitset_name_list, 
                                   subgroup_id_list, 
                                   subgroup_filter_fn, 
                                   cohort_args,
                                   case_id_columns = [])

ds_case_dict

DatasetDict({
    TrulicityRx|0_C1-gF-Ab50: Dataset({
        features: ['PID', 'ObsDT'],
        num_rows: 40517
    })
    TrulicityRx|1_C1-gF-Bl50: Dataset({
        features: ['PID', 'ObsDT'],
        num_rows: 17676
    })
    TrulicityRx|2_C1-gM-Ab50: Dataset({
        features: ['PID', 'ObsDT'],
        num_rows: 32213
    })
    TrulicityRx|3_C1-gM-Bl50: Dataset({
        features: ['PID', 'ObsDT'],
        num_rows: 10543
    })
    TrulicityRx|4_None: Dataset({
        features: ['PID', 'ObsDT'],
        num_rows: 57
    })
})

# [Part 3] Get Case Observation O_ij List

## [Step 1] Case Observations

In [11]:
case_observations

['FutEduTknY:ro.EgmEdu-Af1W_ct.FutRxEduTkn',
 'PDemo:ro.P-Demo_ct.InCaseTkn',
 'PZip3Demo:ro.P-Zip3DemoNume_ct.InCaseTkn',
 'PZip3Econ:ro.P-Zip3EconNume_ct.InCaseTkn',
 'PZip3House:ro.P-Zip3HousingNume_ct.InCaseTkn',
 'PZip3Social:ro.P-Zip3SocialNume_ct.InCaseTkn']

In [12]:
from pprint import pprint 
from recfldtkn.pipeline_case import convert_case_observations_to_co_to_observation
from recfldtkn.pipeline_case import get_RecNameList_and_FldTknList

co_to_CaseObsName, co_to_CaseObsNameInfo = convert_case_observations_to_co_to_observation(case_observations)
# print(co_to_CaseObsName)
pprint(co_to_CaseObsNameInfo, sort_dicts=False)

{'FutEduTknY': {'Record_Observations_List': ['EgmEdu-Af1W'],
                'CaseTkn': 'FutRxEduTkn'},
 'PDemo': {'Record_Observations_List': ['P-Demo'], 'CaseTkn': 'InCaseTkn'},
 'PZip3Demo': {'Record_Observations_List': ['P-Zip3DemoNume'],
               'CaseTkn': 'InCaseTkn'},
 'PZip3Econ': {'Record_Observations_List': ['P-Zip3EconNume'],
               'CaseTkn': 'InCaseTkn'},
 'PZip3House': {'Record_Observations_List': ['P-Zip3HousingNume'],
                'CaseTkn': 'InCaseTkn'},
 'PZip3Social': {'Record_Observations_List': ['P-Zip3SocialNume'],
                 'CaseTkn': 'InCaseTkn'}}


In [13]:
# check whether this information is ready. 
pipeline_info_dict = get_RecNameList_and_FldTknList(co_to_CaseObsNameInfo)
pprint(pipeline_info_dict)

{'CaseTknList': ['InCaseTkn', 'FutRxEduTkn'],
 'CkpdNameList': ['Af1W'],
 'FldTknList': ['P-Zip3EconNume',
                'P-Zip3SocialNume',
                'P-Zip3DemoNume',
                'P-Demo',
                'P-Zip3HousingNume'],
 'RecNameList': ['EgmEdu', 'P']}


## [Step 2] ds_caseobslist_dict

In [14]:
from recfldtkn.pipeline_case import pipeline_to_generate_co_to_CaseObsInfo, get_complete_dataset

record_to_ds_rec = {} # will load RFT from disk
record_to_ds_rec_info = {} # will load RFT from disk
use_caseobs_from_disk = True # will save and load CO to and from disk

d = {}
for case_type, ds_case in ds_case_dict.items():
    print(f'\n\n================ case_type: {case_type} ================')
    co_to_CaseObsInfo = pipeline_to_generate_co_to_CaseObsInfo(ds_case, 
                                                                co_to_CaseObsNameInfo, 
                                                                SPACE, 
                                                                cohort_args,
                                                                case_id_columns, 
                                                                record_to_ds_rec, 
                                                                record_to_ds_rec_info,
                                                                use_caseobs_from_disk,
                                                                batch_size = 1000)
    ds = get_complete_dataset(co_to_CaseObsInfo)
    # print(ds)
    d[case_type] = ds

ds_caseobslist_dict = datasets.DatasetDict(d)



================ case_type: TrulicityRx|0_C1-gF-Ab50 ================
['EgmEdu-Af1W'] FutRxEduTkn


[INFO:2024-02-12 01:18:07,290:(observer.py@342 recfldtkn.observer)]: caseobs_ids num: 40517
Map: 100%|██████████| 40517/40517 [00:01<00:00, 22769.98 examples/s]
[INFO:2024-02-12 01:18:09,131:(observer.py@393 recfldtkn.observer)]: No new calculated caseobs
[INFO:2024-02-12 01:18:09,281:(observer.py@342 recfldtkn.observer)]: caseobs_ids num: 18166


['P-Demo'] InCaseTkn


Map: 100%|██████████| 40517/40517 [00:01<00:00, 23389.00 examples/s]
[INFO:2024-02-12 01:18:11,034:(observer.py@393 recfldtkn.observer)]: No new calculated caseobs
[INFO:2024-02-12 01:18:11,193:(observer.py@342 recfldtkn.observer)]: caseobs_ids num: 18166


['P-Zip3DemoNume'] InCaseTkn


...

Map: 100%|██████████| 57/57 [00:00<00:00, 7549.67 examples/s]
[INFO:2024-02-12 01:18:59,725:(observer.py@393 recfldtkn.observer)]: No new calculated caseobs
[INFO:2024-02-12 01:18:59,742:(observer.py@342 recfldtkn.observer)]: caseobs_ids num: 26


['P-Zip3HousingNume'] InCaseTkn


Map: 100%|██████████| 57/57 [00:00<00:00, 8470.94 examples/s]
[INFO:2024-02-12 01:18:59,762:(observer.py@393 recfldtkn.observer)]: No new calculated caseobs
[INFO:2024-02-12 01:18:59,780:(observer.py@342 recfldtkn.observer)]: caseobs_ids num: 26


['P-Zip3SocialNume'] InCaseTkn


Map: 100%|██████████| 57/57 [00:00<00:00, 7671.77 examples/s]
[INFO:2024-02-12 01:18:59,801:(observer.py@393 recfldtkn.observer)]: No new calculated caseobs


In [15]:
ds_caseobslist_dict

DatasetDict({
    TrulicityRx|0_C1-gF-Ab50: Dataset({
        features: ['PID', 'ObsDT', 'PInvID', 'RxID', 'FutEduTknY_tid', 'FutEduTknY_wgt', 'PDemo_tid', 'PDemo_wgt', 'PZip3Demo_tid', 'PZip3Demo_wgt', 'PZip3Econ_tid', 'PZip3Econ_wgt', 'PZip3House_tid', 'PZip3House_wgt', 'PZip3Social_tid', 'PZip3Social_wgt'],
        num_rows: 40517
    })
    TrulicityRx|1_C1-gF-Bl50: Dataset({
        features: ['PID', 'ObsDT', 'PInvID', 'RxID', 'FutEduTknY_tid', 'FutEduTknY_wgt', 'PDemo_tid', 'PDemo_wgt', 'PZip3Demo_tid', 'PZip3Demo_wgt', 'PZip3Econ_tid', 'PZip3Econ_wgt', 'PZip3House_tid', 'PZip3House_wgt', 'PZip3Social_tid', 'PZip3Social_wgt'],
        num_rows: 17676
    })
    TrulicityRx|2_C1-gM-Ab50: Dataset({
        features: ['PID', 'ObsDT', 'PInvID', 'RxID', 'FutEduTknY_tid', 'FutEduTknY_wgt', 'PDemo_tid', 'PDemo_wgt', 'PZip3Demo_tid', 'PZip3Demo_wgt', 'PZip3Econ_tid', 'PZip3Econ_wgt', 'PZip3House_tid', 'PZip3House_wgt', 'PZip3Social_tid', 'PZip3Social_wgt'],
        num_rows: 32213
    })

## [Step 3] A Look at the Dataset of Oij Observation

In [16]:
case_type = [i for i in ds_caseobslist_dict.keys()][0]
ds_caseobslist = ds_caseobslist_dict[case_type]
ds_caseobslist

Dataset({
    features: ['PID', 'ObsDT', 'PInvID', 'RxID', 'FutEduTknY_tid', 'FutEduTknY_wgt', 'PDemo_tid', 'PDemo_wgt', 'PZip3Demo_tid', 'PZip3Demo_wgt', 'PZip3Econ_tid', 'PZip3Econ_wgt', 'PZip3House_tid', 'PZip3House_wgt', 'PZip3Social_tid', 'PZip3Social_wgt'],
    num_rows: 40517
})

In [17]:
df = ds_caseobslist.to_pandas()
display(HTML(df.tail().to_html()))
ds

,PID,ObsDT,PInvID,RxID,FutEduTknY_tid,FutEduTknY_wgt,PDemo_tid,PDemo_wgt,PZip3Demo_tid,PZip3Demo_wgt,PZip3Econ_tid,PZip3Econ_wgt,PZip3House_tid,PZip3House_wgt,PZip3Social_tid,PZip3Social_wgt
40512,1046197,2023-09-22 14:52:37.577,1046197-001,1046197-001-001,"[1, 4]","[0.0, 0.0]","[2, 7]","[1, 1]","[21, 17, 5, 1, 25, 13, 9, 22, 18, 6, 2, 26, 14, 10, 23, 19, 7, 3, 27, 15, 11]","[6154.44, 6889.88, 1961.38, 9961.0, 4036.12, 8017.44, 414.44, 5052.5, 4872.5, 1373.5, 8061.0, 1648.5, 7172.0, 52.5, 50187144.13, 74417226.12, 4924310.52, 137359413.47, 33138440.52, 101365570.0, 993292.4]","[15, 46, 19, 5, 9, 33, 23, 27, 37, 1, 50, 54, 45, 16, 47, 20, 6, 10, 34, 24, 28, 38, 2, 51, 55, 43, 17, 48, 21, 7, 12, 35, 25, 30, 40, 3, 52, 56, 44]","[5586.25, 410.19, 2913.69, 4599.31, 16.95, 455.38, 1036.5, 52875.38, 61108.0, 11345.88, 7777.0, 5257.5, 1.0, 3731.0, 398.5, 1913.5, 2868.5, 19.0, 293.5, 900.0, 47620.0, 63387.0, 8523.0, 4539.5, 4667.5, 1.0, 55274388.87, 244373.36, 16415363.43, 38339031.96, 23.62, 280295.05, 1166124.0, 616661561.92, 711633186.91, 199029256.52, 116502317.47, 32711240.67, 1.0]","[1, 5, 9, 2, 6, 10, 3, 7, 11]","[6036.62, 5108.31, 952.75, 5194.0, 3888.5, 547.5, 49764602.92, 39674023.83, 1672859.53]","[31, 43, 13, 37, 25, 55, 49, 67, 1, 7, 19, 61, 32, 44, 14, 38, 26, 56, 50, 68, 2, 8, 20, 62, 34, 46, 16, 40, 28, 58, 52, 70, 4, 10, 22, 64]","[5108.31, 1687.19, 282.75, 1039.44, 2098.94, 1856.62, 1807.19, 179.62, 2820.5, 7581.19, 11425.06, 4327.31, 3888.5, 790.5, 253.5, 823.5, 1877.5, 913.0, 887.0, 110.0, 1129.5, 5408.0, 9319.5, 2938.0, 39674023.83, 5923341.5, 94986.6, 1336960.0, 6431701.8, 6584416.12, 6524035.63, 45080.52, 18815843.6, 90252482.43, 182391979.66, 32322706.76]"
40513,1046201,2023-08-04 13:27:41.109,1046201-001,1046201-001-000,"[1, 4]","[0.0, 0.0]","[2, 7]","[1, 1]","[21, 17, 5, 1, 25, 13, 9, 22, 18, 6, 2, 26, 14, 10, 23, 19, 7, 3, 27, 15, 11]","[2911.21, 2975.2, 1176.61, 4636.57, 5268.93, 174.96, 157.96, 1741.0, 1327.0, 501.0, 2352.5, 2704.0, 32.0, 1.5, 18753310.83, 22326869.8, 3655896.9, 49971140.72, 65090678.58, 101408.07, 251018.47]","[15, 46, 19, 5, 9, 33, 23, 27, 37, 1, 50, 54, 45, 16, 47, 20, 6, 10, 34, 24, 28, 38, 2, 51, 55, 43, 17, 48, 21, 7, 12, 35, 25, 30, 40, 3, 52, 56, 44]","[2878.57, 106.3, 1384.84, 2277.84, 24.08, 83.27, 179.34, 68706.61, 84411.43, 5554.38, 4419.36, 2079.89, 1.0, 1261.0, 50.0, 596.0, 908.5, 23.6, 32.0, 83.5, 64896.0, 77297.0, 2433.0, 1757.5, 1154.0, 1.0, 21338144.18, 23833.45, 5239496.76, 13835727.85, 19.77, 13283.62, 60191.25, 301537808.45, 444122731.25, 77930075.84, 56905149.94, 7801291.55, 1.0]","[1, 5, 9, 2, 6, 10, 3, 7, 11]","[2598.5, 2361.57, 492.2, 1163.0, 1069.0, 257.5, 14810956.18, 13726528.43, 514646.31]","[31, 43, 13, 37, 25, 55, 49, 67, 1, 7, 19, 61, 32, 44, 14, 38, 26, 56, 50, 68, 2, 8, 20, 62, 34, 46, 16, 40, 28, 58, 52, 70, 4, 10, 22, 64]","[2361.57, 1279.39, 175.45, 347.07, 559.66, 1357.66, 1313.29, 55.79, 1615.77, 3924.46, 5119.71, 2075.38, 1069.0, 561.0, 103.5, 210.0, 234.0, 649.5, 580.0, 24.5, 445.0, 1759.5, 2597.0, 897.5, 13726528.43, 4438435.62, 52647.27, 225795.67, 939632.74, 4778510.66, 4625401.3, 6259.01, 12217807.27, 39490895.31, 58849841.15, 11163980.09]"
40514,1046201,2023-09-08 18:37:24.059,1046201-002,1046201-002-000,"[1, 4]","[0.0, 0.0]","[2, 7]","[1, 1]","[21, 17, 5, 1, 25, 13, 9, 22, 18, 6, 2, 26, 14, 10, 23, 19, 7, 3, 27, 15, 11]","[2911.21, 2975.2, 1176.61, 4636.57, 5268.93, 174.96, 157.96, 1741.0, 1327.0, 501.0, 2352.5, 2704.0, 32.0, 1.5, 18753310.83, 22326869.8, 3655896.9, 49971140.72, 65090678.58, 101408.07, 251018.47]","[15, 46, 19, 5, 9, 33, 23, 27, 37, 1, 50, 54, 45, 16, 47, 20, 6, 10, 34, 24, 28, 38, 2, 51, 55, 43, 17, 48, 21, 7, 12, 35, 25, 30, 40, 3, 52, 56, 44]","[2878.57, 106.3, 1384.84, 2277.84, 24.08, 83.27, 179.34, 68706.61, 84411.43, 5554.38, 4419.36, 2079.89, 1.0, 1261.0, 50.0, 596.0, 908.5, 23.6, 32.0, 83.5, 64896.0, 77297.0, 2433.0, 1757.5, 1154.0, 1.0, 21338144.18, 23833.45, 5239496.76, 13835727.85, 19.77, 13283.62, 60191.25, 3015378

Dataset({
    features: ['PID', 'ObsDT', 'PInvID', 'RxID', 'FutEduTknY_tid', 'FutEduTknY_wgt', 'PDemo_tid', 'PDemo_wgt', 'PZip3Demo_tid', 'PZip3Demo_wgt', 'PZip3Econ_tid', 'PZip3Econ_wgt', 'PZip3House_tid', 'PZip3House_wgt', 'PZip3Social_tid', 'PZip3Social_wgt'],
    num_rows: 57
})

# [Part 5] fn Case Task Operation

## [Step 1]: $\Gamma$ CaseTaskOp

In [18]:
# Gamma
CaseTaskOp = args.case_taskop 

# CO_to_COvocab
co_to_CaseObsVocab = {co: CaseObsInfo['vocab_caseobs'] for co, CaseObsInfo in co_to_CaseObsInfo.items()}
co_to_CaseObsVocab

{'FutEduTknY': {'tkn': {'tid2tkn': {0: 'unk',
    1: 'InvBtnEgm',
    2: 'InvBtnEgmClicks',
    3: 'InvMinsUntil1stEgm',
    4: 'RxBtnEgm',
    5: 'RxBtnEgmClicks',
    6: 'RxMinsUntil1stEgm'},
   'tkn2tid': {'unk': 0,
    'InvBtnEgm': 1,
    'InvBtnEgmClicks': 2,
    'InvMinsUntil1stEgm': 3,
    'RxBtnEgm': 4,
    'RxBtnEgmClicks': 5,
    'RxMinsUntil1stEgm': 6}}},
 'PDemo': {'tkn': {'tid2tkn': {0: 'patient_gender_unk',
    1: 'patient_gender_minor',
    2: 'patient_gender_F',
    3: 'patient_gender_M',
    4: 'patient_gender_U',
    5: 'patient_age_bucket_unk',
    6: 'patient_age_bucket_minor',
    7: 'patient_age_bucket_51-60',
    8: 'patient_age_bucket_61-70',
    9: 'patient_age_bucket_41-50',
    10: 'patient_age_bucket_71-80',
    11: 'patient_age_bucket_31-40',
    12: 'patient_age_bucket_18-30',
    13: 'patient_age_bucket_81+'},
   'tkn2tid': {'patient_gender_unk': 0,
    'patient_gender_minor': 1,
    'patient_gender_F': 2,
    'patient_gender_M': 3,
    'patient_gender_U'

## [Step 2]* CaseOp Vocab [TO Dev]

In [19]:
import inspect

##################################
BATCHED = '1'
##################################

##################################
def get_case_op_vocab(co_to_CaseObsVocab):
    CaseTaskOpVocab = {}
    co_Y_list = [i for i in co_to_CaseObsVocab if 'TknY' in i]
    co_X_list = [i for i in co_to_CaseObsVocab if 'TknY' not in i]

    sequence_name = 'input_ids'
    idx2tkn_all = []
    for co in co_X_list:
        CaseObsVocab = co_to_CaseObsVocab[co] 
        idx2tkn = [co+':'+ tkn for tid, tkn in CaseObsVocab['tkn']['tid2tkn'].items()]
        idx2tkn_all = idx2tkn_all + idx2tkn
    tid2tkn_all = {i: tkn for i, tkn in enumerate(idx2tkn_all)}
    tkn2tid_all = {tkn: i for i, tkn in enumerate(idx2tkn_all)}
    CaseTaskOpVocab[sequence_name] = {'tid2tkn': tid2tkn_all, 'tkn2tid': tkn2tid_all}
    
    if len(co_Y_list) == 1:
        sequence_name = 'labels'
        co_Y = co_Y_list[0]
        co = co_Y
        CaseObsVocab = co_to_CaseObsVocab[co]
        idx2tkn_all = [co+':'+ tkn for tid, tkn in CaseObsVocab['tkn']['tid2tkn'].items()]
        tid2tkn_all = {i: tkn for i, tkn in enumerate(idx2tkn_all)}
        tkn2tid_all = {tkn: i for i, tkn in enumerate(idx2tkn_all)}
        CaseTaskOpVocab[sequence_name] = {'tid2tkn': tid2tkn_all, 'tkn2tid': tkn2tid_all}
    
    return CaseTaskOpVocab
##################################

get_case_op_vocab.fn_string = inspect.getsource(get_case_op_vocab)

In [20]:
CaseTaskOpVocab = get_case_op_vocab(co_to_CaseObsVocab)
for seqname, vocab in CaseTaskOpVocab.items():
    print(seqname)
    print(len(vocab['tid2tkn']))

input_ids
187
labels
7


## [Step 3]* fn_Case_Operation_Tkn $\Gamma$. (To Dev)

In [21]:
subsetname = [i for i in ds_caseobslist_dict][0]
subsetname

'TrulicityRx|0_C1-gF-Ab50'

In [22]:
ds_caseobslist = ds_caseobslist_dict[subsetname]
print(ds_caseobslist)

Dataset({
    features: ['PID', 'ObsDT', 'PInvID', 'RxID', 'FutEduTknY_tid', 'FutEduTknY_wgt', 'PDemo_tid', 'PDemo_wgt', 'PZip3Demo_tid', 'PZip3Demo_wgt', 'PZip3Econ_tid', 'PZip3Econ_wgt', 'PZip3House_tid', 'PZip3House_wgt', 'PZip3Social_tid', 'PZip3Social_wgt'],
    num_rows: 40517
})


In [23]:
examples = ds_caseobslist[:10]
examples

{'PID': [1000002,
  1000003,
  1000004,
  1000004,
  1000006,
  1000006,
  1000006,
  1000006,
  1000006,
  1000006],
 'ObsDT': [Timestamp('2023-09-21 23:38:11.348000'),
  Timestamp('2023-09-05 16:33:52.699000'),
  Timestamp('2023-05-09 20:46:13.911000'),
  Timestamp('2023-09-01 17:07:57.276000'),
  Timestamp('2023-03-28 16:58:46.211000'),
  Timestamp('2023-04-14 18:10:32.633000'),
  Timestamp('2023-05-12 16:07:29.402000'),
  Timestamp('2023-06-27 15:52:15.690000'),
  Timestamp('2023-08-15 19:31:23.678000'),
  Timestamp('2023-09-14 16:21:28.061000')],
 'PInvID': ['1000002-000',
  '1000003-000',
  '1000004-003',
  '1000004-007',
  '1000006-002',
  '1000006-004',
  '1000006-005',
  '1000006-009',
  '1000006-013',
  '1000006-017'],
 'RxID': ['1000002-000-000',
  '1000003-000-000',
  '1000004-003-000',
  '1000004-007-000',
  '1000006-002-000',
  '1000006-004-000',
  '1000006-005-000',
  '1000006-009-000',
  '1000006-013-000',
  '1000006-017-002'],
 'FutEduTknY_tid': [[1, 4],
  [1, 4],
  [1

In [24]:
##################################
def fn_Case_Operation_Tkn(examples, co_to_CaseObsVocab, CaseTaskOpVocab):
    num_examples = len(examples[[i for i in examples.keys()][0]])
    
    idx_to_examples = {i: {k: v[i] for k, v in examples.items()} for i in range(num_examples)}

    co_Y_list = [i for i in co_to_CaseObsVocab if 'TknY' in i]
    co_X_list = [i for i in co_to_CaseObsVocab if 'TknY' not in i]
    # print(co_Y)
    # print(co_X_list)
    
    if len(co_Y_list) == 1:
        results = {'input_ids': [], 'input_wgts': [], 'labels': [], 'label_wgts': []}
    else:
        results = {'input_ids': [], 'input_wgts': []}

    for idx, example in idx_to_examples.items():
        result_case = {}
        # ============================================ # 
        # get input_ids and input_wgts
        sequence_name = 'input_ids' 
        tkn2tid = CaseTaskOpVocab[sequence_name]['tkn2tid']

        X_tid_updated_total = []
        X_wgt_total = []
        for co in co_X_list:
            CaseObsVocab = co_to_CaseObsVocab[co]
            X_tid = example[co +'_tid']
            X_tkn = [co + ':' + CaseObsVocab['tkn']['tid2tkn'][tid] for tid in X_tid]
            X_tid_updated = [tkn2tid[i] for i in X_tkn]
            X_wgt = example[co +'_wgt']
            X_tid_updated_total = X_tid_updated_total + X_tid_updated
            X_wgt_total = X_wgt_total + X_wgt
        
        # version 1: for DL and ML
        result_case['input_ids'] = X_tid_updated_total
        result_case['input_wgts'] = X_wgt_total

        # version 2: for ML
        # tid2wgt = dict(zip(X_tid_updated_total, X_wgt_total))
        # whole_ids = list(range(len(tkn2tid)))
        # result_case['input_ids'] = whole_ids
        # result_case['input_wgts'] = [tid2wgt.get(tid, 0) for tid in whole_ids]

        # ============================================ # 
        if len(co_Y_list) == 1:
            co_Y = co_Y_list[0]
            # get labels and label_wgts
            sequence_name = 'labels'
            tkn2tid = CaseTaskOpVocab[sequence_name]['tkn2tid']
            CaseObsVocab = co_to_CaseObsVocab[co_Y]
            Y_tid = example[co_Y+'_tid']
            Y_tkn = [co_Y + ':' + CaseObsVocab['tkn']['tid2tkn'][tid] for tid in Y_tid]
            Y_tid_updated = [CaseTaskOpVocab[sequence_name]['tkn2tid'][i] for i in Y_tkn]
            Y_wgt = example[co_Y+'_wgt']

            # version 1: for DL and ML
            result_case['labels'] = Y_tid_updated
            result_case['label_wgts'] = Y_wgt

            # version 2: for ML (consume too much memory and time)
            # tid2wgt = dict(zip(Y_tid_updated, Y_wgt))
            # whole_ids = list(range(len(tkn2tid)))
            # result_case['labels'] = whole_ids
            # result_case['label_wgts'] = [tid2wgt.get(tid, 0) for tid in whole_ids]

        for k, v in result_case.items(): results[k].append(v)
        
    return results 
##################################

fn_Case_Operation_Tkn.fn_string = inspect.getsource(fn_Case_Operation_Tkn)

In [25]:
import numpy as np 
results = fn_Case_Operation_Tkn(examples, co_to_CaseObsVocab, CaseTaskOpVocab)
for k, v in results.items(): 
    print(np.array([len(i) for i in v]))
    print(k, v) # <-- you might meet the error. 
    results[k] = v

[107   6 107 107 107 107 107 107 107 107]
input_ids [[2, 7, 35, 31, 19, 15, 39, 27, 23, 36, 32, 20, 16, 40, 28, 24, 37, 33, 21, 17, 41, 29, 25, 58, 89, 62, 48, 52, 76, 66, 70, 80, 44, 93, 97, 88, 59, 90, 63, 49, 53, 77, 67, 71, 81, 45, 94, 98, 86, 60, 91, 64, 50, 55, 78, 68, 73, 83, 46, 95, 99, 87, 102, 106, 110, 103, 107, 111, 104, 108, 112, 145, 157, 127, 151, 139, 169, 163, 181, 115, 121, 133, 175, 146, 158, 128, 152, 140, 170, 164, 182, 116, 122, 134, 176, 148, 160, 130, 154, 142, 172, 166, 184, 118, 124, 136, 178], [2, 10, 14, 43, 101, 114], [2, 10, 35, 31, 19, 15, 39, 27, 23, 36, 32, 20, 16, 40, 28, 24, 37, 33, 21, 17, 41, 29, 25, 58, 89, 62, 48, 52, 76, 66, 70, 80, 44, 93, 97, 88, 59, 90, 63, 49, 53, 77, 67, 71, 81, 45, 94, 98, 86, 60, 91, 64, 50, 55, 78, 68, 73, 83, 46, 95, 99, 87, 102, 106, 110, 103, 107, 111, 104, 108, 112, 145, 157, 127, 151, 139, 169, 163, 181, 115, 121, 133, 175, 146, 158, 128, 152, 140, 170, 164, 182, 116, 122, 134, 176, 148, 160, 130, 154, 142, 172, 166,

In [26]:
# CaseTaskOpVocab['input_ids']['tid2tkn']

In [27]:
results

{'input_ids': [[2,
   7,
   35,
   31,
   19,
   15,
   39,
   27,
   23,
   36,
   32,
   20,
   16,
   40,
   28,
   24,
   37,
   33,
   21,
   17,
   41,
   29,
   25,
   58,
   89,
   62,
   48,
   52,
   76,
   66,
   70,
   80,
   44,
   93,
   97,
   88,
   59,
   90,
   63,
   49,
   53,
   77,
   67,
   71,
   81,
   45,
   94,
   98,
   86,
   60,
   91,
   64,
   50,
   55,
   78,
   68,
   73,
   83,
   46,
   95,
   99,
   87,
   102,
   106,
   110,
   103,
   107,
   111,
   104,
   108,
   112,
   145,
   157,
   127,
   151,
   139,
   169,
   163,
   181,
   115,
   121,
   133,
   175,
   146,
   158,
   128,
   152,
   140,
   170,
   164,
   182,
   116,
   122,
   134,
   176,
   148,
   160,
   130,
   154,
   142,
   172,
   166,
   184,
   118,
   124,
   136,
   178],
  [2, 10, 14, 43, 101, 114],
  [2,
   10,
   35,
   31,
   19,
   15,
   39,
   27,
   23,
   36,
   32,
   20,
   16,
   40,
   28,
   24,
   37,
   33,
   21,
   17,
   41,
   29,
   25,
   58

In [28]:
results['input_ids']

[[2,
  7,
  35,
  31,
  19,
  15,
  39,
  27,
  23,
  36,
  32,
  20,
  16,
  40,
  28,
  24,
  37,
  33,
  21,
  17,
  41,
  29,
  25,
  58,
  89,
  62,
  48,
  52,
  76,
  66,
  70,
  80,
  44,
  93,
  97,
  88,
  59,
  90,
  63,
  49,
  53,
  77,
  67,
  71,
  81,
  45,
  94,
  98,
  86,
  60,
  91,
  64,
  50,
  55,
  78,
  68,
  73,
  83,
  46,
  95,
  99,
  87,
  102,
  106,
  110,
  103,
  107,
  111,
  104,
  108,
  112,
  145,
  157,
  127,
  151,
  139,
  169,
  163,
  181,
  115,
  121,
  133,
  175,
  146,
  158,
  128,
  152,
  140,
  170,
  164,
  182,
  116,
  122,
  134,
  176,
  148,
  160,
  130,
  154,
  142,
  172,
  166,
  184,
  118,
  124,
  136,
  178],
 [2, 10, 14, 43, 101, 114],
 [2,
  10,
  35,
  31,
  19,
  15,
  39,
  27,
  23,
  36,
  32,
  20,
  16,
  40,
  28,
  24,
  37,
  33,
  21,
  17,
  41,
  29,
  25,
  58,
  89,
  62,
  48,
  52,
  76,
  66,
  70,
  80,
  44,
  93,
  97,
  88,
  59,
  90,
  63,
  49,
  53,
  77,
  67,
  71,
  81,
  45,
  94,
  98,

In [29]:
results['input_wgts']

[[1,
  1,
  3990.67,
  4201.75,
  1383.89,
  6303.05,
  5898.65,
  1781.81,
  48.49,
  3142.0,
  2638.0,
  996.0,
  4191.0,
  3623.0,
  344.0,
  0.0,
  13312654.12,
  15744970.08,
  1681750.1,
  33545106.12,
  31013867.34,
  11653142.69,
  8899.11,
  3623.39,
  204.02,
  1740.65,
  2957.21,
  31.24,
  182.49,
  335.74,
  58013.75,
  69691.84,
  7288.98,
  5455.04,
  2922.95,
  1.0,
  2081.0,
  148.0,
  997.0,
  1759.0,
  29.3,
  93.0,
  208.0,
  55341.0,
  68477.0,
  4569.0,
  3311.0,
  2408.0,
  1.0,
  12408356.35,
  53466.87,
  2966132.37,
  8259036.38,
  50.25,
  56606.25,
  235440.59,
  229509420.11,
  262558033.09,
  45630322.73,
  27618317.25,
  7791731.98,
  1.0,
  3531.82,
  3037.28,
  924.51,
  2651.0,
  1895.0,
  608.0,
  10692385.11,
  8283993.1,
  648325.58,
  3037.28,
  1528.67,
  145.77,
  509.16,
  853.68,
  1634.65,
  1637.53,
  96.54,
  1210.82,
  4928.07,
  7355.0,
  2553.25,
  1895.0,
  917.0,
  99.0,
  314.0,
  436.0,
  921.0,
  1064.0,
  70.0,
  548.0,
  2833.0,
  

In [30]:
results['labels']

[[1, 4],
 [1, 4],
 [1, 4],
 [1, 4],
 [1, 4],
 [1, 4],
 [1, 4],
 [1, 4],
 [1, 4],
 [1, 4]]

In [31]:
results['label_wgts']

[[0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0]]

## Save to Files

In [32]:
from recfldtkn.loadtools import convert_variables_to_pystirng, load_module_variables

prefix = ['import pandas as pd', 'import numpy as np']
string_variables = [BATCHED]
fn_variables = [get_case_op_vocab, fn_Case_Operation_Tkn]
pycode = convert_variables_to_pystirng(string_variables = string_variables, 
                                       fn_variables = fn_variables, 
                                       prefix = prefix)

pypath = os.path.join(SPACE['CODE_FN'], 'fn_casetaskop', f'casetaskop_{CaseTaskOp}.py')
with open(pypath, 'w') as file: file.write(pycode)

# Create a HTML link and display it
full_path = os.path.join(WORKSPACE_PATH, pypath)
display(HTML(f'{pypath} <a href="{full_path}" target="_blank">Open File</a>'))

# Examples

## A Few Examples

In [33]:
co = [i for i in ds_caseobslist_dict.keys()][0]
ds_caseobslist = ds_caseobslist_dict[case_type]
ds_caseobslist

Dataset({
    features: ['PID', 'ObsDT', 'PInvID', 'RxID', 'FutEduTknY_tid', 'FutEduTknY_wgt', 'PDemo_tid', 'PDemo_wgt', 'PZip3Demo_tid', 'PZip3Demo_wgt', 'PZip3Econ_tid', 'PZip3Econ_wgt', 'PZip3House_tid', 'PZip3House_wgt', 'PZip3Social_tid', 'PZip3Social_wgt'],
    num_rows: 40517
})

In [34]:
examples = ds_caseobslist[:10]
examples

{'PID': [1000002,
  1000003,
  1000004,
  1000004,
  1000006,
  1000006,
  1000006,
  1000006,
  1000006,
  1000006],
 'ObsDT': [Timestamp('2023-09-21 23:38:11.348000'),
  Timestamp('2023-09-05 16:33:52.699000'),
  Timestamp('2023-05-09 20:46:13.911000'),
  Timestamp('2023-09-01 17:07:57.276000'),
  Timestamp('2023-03-28 16:58:46.211000'),
  Timestamp('2023-04-14 18:10:32.633000'),
  Timestamp('2023-05-12 16:07:29.402000'),
  Timestamp('2023-06-27 15:52:15.690000'),
  Timestamp('2023-08-15 19:31:23.678000'),
  Timestamp('2023-09-14 16:21:28.061000')],
 'PInvID': ['1000002-000',
  '1000003-000',
  '1000004-003',
  '1000004-007',
  '1000006-002',
  '1000006-004',
  '1000006-005',
  '1000006-009',
  '1000006-013',
  '1000006-017'],
 'RxID': ['1000002-000-000',
  '1000003-000-000',
  '1000004-003-000',
  '1000004-007-000',
  '1000006-002-000',
  '1000006-004-000',
  '1000006-005-000',
  '1000006-009-000',
  '1000006-013-000',
  '1000006-017-002'],
 'FutEduTknY_tid': [[1, 4],
  [1, 4],
  [1

In [35]:
results = fn_Case_Operation_Tkn(examples, co_to_CaseObsVocab, CaseTaskOpVocab)
for k, v in results.items():
    print(k, v)

input_ids [[2, 7, 35, 31, 19, 15, 39, 27, 23, 36, 32, 20, 16, 40, 28, 24, 37, 33, 21, 17, 41, 29, 25, 58, 89, 62, 48, 52, 76, 66, 70, 80, 44, 93, 97, 88, 59, 90, 63, 49, 53, 77, 67, 71, 81, 45, 94, 98, 86, 60, 91, 64, 50, 55, 78, 68, 73, 83, 46, 95, 99, 87, 102, 106, 110, 103, 107, 111, 104, 108, 112, 145, 157, 127, 151, 139, 169, 163, 181, 115, 121, 133, 175, 146, 158, 128, 152, 140, 170, 164, 182, 116, 122, 134, 176, 148, 160, 130, 154, 142, 172, 166, 184, 118, 124, 136, 178], [2, 10, 14, 43, 101, 114], [2, 10, 35, 31, 19, 15, 39, 27, 23, 36, 32, 20, 16, 40, 28, 24, 37, 33, 21, 17, 41, 29, 25, 58, 89, 62, 48, 52, 76, 66, 70, 80, 44, 93, 97, 88, 59, 90, 63, 49, 53, 77, 67, 71, 81, 45, 94, 98, 86, 60, 91, 64, 50, 55, 78, 68, 73, 83, 46, 95, 99, 87, 102, 106, 110, 103, 107, 111, 104, 108, 112, 145, 157, 127, 151, 139, 169, 163, 181, 115, 121, 133, 175, 146, 158, 128, 152, 140, 170, 164, 182, 116, 122, 134, 176, 148, 160, 130, 154, 142, 172, 166, 184, 118, 124, 136, 178], [2, 10, 35, 31,

In [36]:
# ############################
# idx2tkn = [tkn for tid, tkn in CaseTaskOpVocab['input_ids']['tid2tkn'].items()]
# df_input = pd.DataFrame(results['input_wgts'], columns = idx2tkn)
# df_input

In [37]:
# ############################
# idx2tkn = [tkn for tid, tkn in CaseTaskOpVocab['labels']['tid2tkn'].items()]
# df_label = pd.DataFrame(results['label_wgts'], columns = idx2tkn)
# df_label

# Gamma Pipeline

In [38]:
ds_case_dict

DatasetDict({
    TrulicityRx|0_C1-gF-Ab50: Dataset({
        features: ['PID', 'ObsDT', 'PInvID', 'RxID'],
        num_rows: 40517
    })
    TrulicityRx|1_C1-gF-Bl50: Dataset({
        features: ['PID', 'ObsDT', 'PInvID', 'RxID'],
        num_rows: 17676
    })
    TrulicityRx|2_C1-gM-Ab50: Dataset({
        features: ['PID', 'ObsDT', 'PInvID', 'RxID'],
        num_rows: 32213
    })
    TrulicityRx|3_C1-gM-Bl50: Dataset({
        features: ['PID', 'ObsDT', 'PInvID', 'RxID'],
        num_rows: 10543
    })
    TrulicityRx|4_None: Dataset({
        features: ['PID', 'ObsDT', 'PInvID', 'RxID'],
        num_rows: 57
    })
})

In [39]:
from recfldtkn.pipeline_case import pipeline_case

results = pipeline_case(ds_case_dict,     # C
                        case_observations,# CO 
                        CaseTaskOp,       # Gamma
                        case_id_columns,
                        cohort_args, 
                        record_to_ds_rec, 
                        record_to_ds_rec_info,
                        use_caseobs_from_disk,
                        SPACE)



================ case_type: TrulicityRx|0_C1-gF-Ab50 ================
['EgmEdu-Af1W'] FutRxEduTkn


[INFO:2024-02-12 01:19:00,385:(observer.py@342 recfldtkn.observer)]: caseobs_ids num: 40517
Map: 100%|██████████| 40517/40517 [00:01<00:00, 23136.15 examples/s]
[INFO:2024-02-12 01:19:02,186:(observer.py@393 recfldtkn.observer)]: No new calculated caseobs
[INFO:2024-02-12 01:19:02,326:(observer.py@342 recfldtkn.observer)]: caseobs_ids num: 18166


['P-Demo'] InCaseTkn


Map: 100%|██████████| 40517/40517 [00:01<00:00, 23940.03 examples/s]
[INFO:2024-02-12 01:19:04,036:(observer.py@393 recfldtkn.observer)]: No new calculated caseobs
[INFO:2024-02-12 01:19:04,202:(observer.py@342 recfldtkn.observer)]: caseobs_ids num: 18166


['P-Zip3DemoNume'] InCaseTkn


...

[INFO:2024-02-12 01:19:53,903:(observer.py@342 recfldtkn.observer)]: caseobs_ids num: 26
Map: 100%|██████████| 57/57 [00:00<00:00, 5551.89 examples/s]
[INFO:2024-02-12 01:19:53,934:(observer.py@393 recfldtkn.observer)]: No new calculated caseobs


['P-Zip3SocialNume'] InCaseTkn


[INFO:2024-02-12 01:19:53,958:(observer.py@342 recfldtkn.observer)]: caseobs_ids num: 26
Map: 100%|██████████| 57/57 [00:00<00:00, 6842.85 examples/s]
[INFO:2024-02-12 01:19:53,984:(observer.py@393 recfldtkn.observer)]: No new calculated caseobs


PyPath: ../pipeline/fn_casetaskop/casetaskop_EduRxMLPred.py


Map: 100%|██████████| 57/57 [00:00<00:00, 5818.62 examples/s]


## [Step] A Check

In [40]:
ds_case_proc = results['ds_case_proc']
ds_case_proc

DatasetDict({
    TrulicityRx|0_C1-gF-Ab50: Dataset({
        features: ['input_ids', 'input_wgts', 'labels', 'label_wgts'],
        num_rows: 40517
    })
    TrulicityRx|1_C1-gF-Bl50: Dataset({
        features: ['input_ids', 'input_wgts', 'labels', 'label_wgts'],
        num_rows: 17676
    })
    TrulicityRx|2_C1-gM-Ab50: Dataset({
        features: ['input_ids', 'input_wgts', 'labels', 'label_wgts'],
        num_rows: 32213
    })
    TrulicityRx|3_C1-gM-Bl50: Dataset({
        features: ['input_ids', 'input_wgts', 'labels', 'label_wgts'],
        num_rows: 10543
    })
    TrulicityRx|4_None: Dataset({
        features: ['input_ids', 'input_wgts', 'labels', 'label_wgts'],
        num_rows: 57
    })
})

In [41]:
CaseTaskOpVocab = results['CaseTaskOpVocab']
# CaseTaskOpVocab

In [42]:
subcase_name = [i for i in ds_case_proc][0]
subcase_name

'TrulicityRx|0_C1-gF-Ab50'

In [43]:
ds_ai = ds_case_proc[subcase_name]
ds_ai

Dataset({
    features: ['input_ids', 'input_wgts', 'labels', 'label_wgts'],
    num_rows: 40517
})

In [44]:
ds_ai['labels']
ds_ai['label_wgts']

[[0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [1.0, 1.0, 6.03, 1.0, 1.0, 6.03],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0

In [45]:
df_ai = ds_ai.to_pandas()
df_ai

,input_ids,input_wgts,labels,label_wgts
0,"[2, 7, 35, 31, 19, 15, 39, 27, 23, 36, 32, 20,...","[1.0, 1.0, 3990.67, 4201.75, 1383.89, 6303.05,...","[1, 4]","[0.0, 0.0]"
1,"[2, 10, 14, 43, 101, 114]","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0]","[1, 4]","[0.0, 0.0]"
2,"[2, 10, 35, 31, 19, 15, 39, 27, 23, 36, 32, 20...","[1.0, 1.0, 1726.41, 1668.52, 810.89, 2776.62, ...","[1, 4]","[0.0, 0.0]"
3,"[2, 10, 35, 31, 19, 15, 39, 27, 23, 36, 32, 20...","[1.0, 1.0, 1726.41, 1668.52, 810.89, 2776.62, ...","[1, 4]","[0.0, 0.0]"
4,"[2, 7, 35, 31, 19, 15, 39, 27, 23, 36, 32, 20,...","[1.0, 1.0, 33997.42, 37376.71, 10070.11, 55003...","[1, 4]","[0.0, 0.0]"
...,...,...,...,...
40512,"[2, 7, 35, 31, 19, 15, 39, 27, 23, 36, 32, 20,...","[1.0, 1.0, 6154.44, 6889.88, 1961.38, 9961.0, ...","[1, 4]","[0.0, 0.0]"
40513,"[2, 7, 35, 31, 19, 15, 39, 27, 23, 36, 32, 20,...","[1.0, 1.0, 2911.21, 2975.2, 1176.61, 4636.57, ...","[1, 4]","[0.0, 0.0]"
40514,"[2, 7, 35, 31, 19, 15, 39, 27, 23, 36, 32, 20,...","[1.0, 1.0, 2911.21, 2975.2, 1176.61, 4636.57, ...","[1, 4]","[0.0, 0.0]"
40515,"[2, 7, 35, 31, 19, 15, 39, 27, 23, 36, 32, 20,...","[1.0, 1.0, 2911.21, 2975.2, 1176.61, 4636.57, ...","[1, 4]","[0.0, 0.0]"


In [46]:
s = df_ai['labels'].apply(lambda x: len(x)) == 6
s.mean()

0.02959251672137621

In [ ]:
# ds_caseobslist_dict['TrulicityRx|0_C1-gF-Ab50'][1]

I have two columns which will be further processed for XGBoost model for prediction. 

The input_ids column are the feature location list. For example: [2, 10, 29, 30, 79]. And in the input_wgts columns, it shows the feature value for the corresponding feature location, for example: [0.1, 2, 0.11, 0.33, 0.45]. The feature list's length might be different for each data point (i.e., each row). How do we convert it as the X for XGBoost training?


In [ ]:
from scipy.sparse import csr_matrix

# Adjusting the 'input_ids' to be zero-indexed
min_id = df_ai['input_ids'].apply(lambda x: min(x)).min()
df_ai['input_ids'] = df_ai['input_ids'].apply(lambda x: [i - min_id for i in x])

row_indices = []
col_indices = []
data = []

# Populate row_indices, col_indices, and data
for i, (input_ids, input_wgts) in enumerate(zip(df_ai['input_ids'], df_ai['input_wgts'])):
    row_indices.extend([i] * len(input_ids))  # Repeat the row index for each feature
    col_indices.extend(input_ids)
    data.extend(input_wgts)

num_features = max(col_indices) + 1  # 0-based indexing, add 1 for the actual size
X = csr_matrix((data, (row_indices, col_indices)), shape=(len(df_ai), num_features))

In [ ]:
import xgboost as xgb
from sklearn.model_selection import train_test_split

# define which label to predict, for example if want to predict label 1   
def extract_label_value(labels, weights):
    labels_list = labels.tolist() 
    if 1 in labels_list:
        index = labels_list.index(1)
        return weights[index]
    else:
        return None

y = df_ai.apply(lambda row: extract_label_value(row['labels'], row['label_wgts']), axis=1).values
X_filtered = X[y != None]
y_filtered = y[y != None]


In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
dtrain = xgb.DMatrix(X_train, label=y_train)
dval = xgb.DMatrix(X_val, label=y_val)


params = {
    'max_depth': 3,
    'eta': 0.1,
    'objective': 'binary:logistic',  
    'eval_metric': 'logloss',  
}
num_boost_round = 400

evals_result = {}
bst = xgb.train(params, dtrain, num_boost_round, evals=[(dtrain, 'train'), (dval, 'val')], evals_result=evals_result, verbose_eval=True)

print("Evaluation Results:", evals_result)


import matplotlib.pyplot as plt

epochs = len(evals_result['train']['logloss'])
x_axis = range(0, epochs)

fig, ax = plt.subplots()
ax.plot(x_axis, evals_result['train']['logloss'], label='Train')
ax.plot(x_axis, evals_result['val']['logloss'], label='Validation')
ax.legend()
plt.ylabel('Log Loss')
plt.title('XGBoost Log Loss')
plt.show()


In [ ]:
# df_label['FutEduTknY:RxBtnEgm'].describe()

# Post Process

## Saving to File

In [ ]:
case_taskop = args.case_taskop

for case_split_type, ds_case in ds_case_proc.items():
    case_type, group_name = case_split_type.split('|')
    path = os.path.join(SPACE['MODEL_TASK'], case_taskop, 'AIDataset', case_type, group_name)
    # if not os.path.exists(os.path.dirname(path)): os.makedirs(os.path.dirname(path))
    print(path)
    # ds_case = ds_case.select_columns(new_columns)
    print(ds_case)
    # ds_case.save_to_disk(path)
    # datasets.load_from_disk(path)

In [ ]:
ds_case